In [2]:
import re, pickle
from collections import Counter, defaultdict

import psycopg2
from psycopg2.extras import execute_values
from tqdm import tqdm
from datasets import load_dataset

from utils import strip_punc

/home/cthewolf/Installations/anaconda3/envs/clyde/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds = load_dataset("wikimedia/wikipedia", "20231101.en")['train']

In [4]:
with open('stopwords-en.txt', mode='r') as f:
    stopwords = set([line.strip('\n') for line in f.readlines()])

In [ ]:
total_counts = Counter()
for article in tqdm(ds):
    total_counts.update(re.split(r'[\n| ]+', strip_punc(article['text']).lower()))

In [ ]:
with open('total_counts.pkl', mode='wb') as f:
    pickle.dump(total_counts, f)

In [ ]:
# uncomment to restart from this point
# with open('total_counts.pkl', mode='rb') as f:
#     total_counts = pickle.load(f)

In [ ]:
total = total_counts.total()
inverse_frequencies = {word: round(total / count) for word, count in total_counts.items()
                        if word not in stopwords}

In [ ]:
with open('inverse_frequencies.pkl', mode='wb') as f:
    pickle.dump(inverse_frequencies, f)

In [3]:
# uncomment to restart from this point
with open('inverse_frequencies.pkl', mode='rb') as f:
    inverse_frequencies = pickle.load(f)

In [ ]:
MIN_LENGTH = 20
NUM_KEYWORDS = 10
def reweight(counts):
    return Counter({k: v * inverse_frequencies[k] for k, v in counts.items()
                    if k not in stopwords})

conn = psycopg2.connect("dbname=postgres user=postgres")
cur = conn.cursor()
for article in tqdm(ds):
    # collect keywords for each chunk in the article
    chunks = [para.split(' ') for para in strip_punc(article['text']).lower().split('\n')]
    counters = [reweight(Counter(chunk)) for chunk in chunks]
    wikichunks_rows = []
    keywords_by_chunk = []
    for i, counter in enumerate(counters):
        if len(chunks[i]) < MIN_LENGTH:
            continue
        keywords_by_chunk.append([k for k, _ in counter.most_common(NUM_KEYWORDS)])
        wikichunks_rows.append((article['id'], i, article['title']))

    # add data to wikichunks table
    wikichunks_query = '''
        INSERT INTO wikichunks
        VALUES %s
        RETURNING id;
    '''
    chunk_ids = execute_values(cur, wikichunks_query, wikichunks_rows, '(DEFAULT, %s, %s, %s)', fetch=True)

    # map keywords to chunks
    keywords_rows = defaultdict(list)
    for (chunk_id,), keywords in zip(chunk_ids, keywords_by_chunk):
        for word in keywords:
            keywords_rows[word].append(chunk_id)
    keywords_query = '''
        INSERT INTO keywords
        VALUES %s
        ON CONFLICT (word_hash)
        DO UPDATE SET chunks = keywords.chunks || EXCLUDED.chunks;
    '''
    execute_values(cur, keywords_query, [(hash(k), k, v) for k, v in keywords_rows.items()])
    conn.commit()
cur.close()
conn.close()

  0%|          | 31142/6407814 [04:31<16:26:23, 107.74it/s]